# Loading of data needed for examples

This notebook can be used to load the data into Snowflake that is used for some of the demos.

It will create tables in the database and schema used in the connection parameters ie creds.json

In [1]:
# Snowpark modules
from snowflake.snowpark.session import Session
import snowflake.snowpark.types as T

In [2]:
# Additional modules
import json

In [3]:
# Get a nicer output from .show()
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

Connect to Snowflake

In [4]:
with open('../creds.json') as f:
    connection_parameters = json.load(f)
    
session = Session.builder.configs(connection_parameters).create()

Path to where the data files is and what stage to upload the files to

In [5]:
data_path = "../data/"
data_stage_name = "~" # Using the user stage

### Titanic data

In [ ]:
# Upload the source file to the stage
putResult = session.file.put(f"{data_path}titanic.csv", f"@{data_stage_name}", auto_compress=True, overwrite=True)

In [ ]:
titanicSchema = T.StructType(
    [
        T.StructField("PCLASS", T.IntegerType()),
        T.StructField("SURVIVED", T.StringType()),
        T.StructField("NAME", T.StringType()),
        T.StructField("SEX", T.StringType()),
        T.StructField("AGE", T.FloatType()),
        T.StructField("SIBSP", T.FloatType()),
        T.StructField("PARCH", T.FloatType()),
        T.StructField("TICKET", T.StringType()),
        T.StructField("FARE", T.FloatType()),
        T.StructField("CABIN", T.StringType()),
        T.StructField("EMBARKED", T.StringType()),
        T.StructField("BOAT", T.StringType()),
        T.StructField("BODY", T.IntegerType()),
        T.StructField("HOME_DEST", T.StringType()),
    ]
)

# Crete a reader
dfReader = session.read.schema(titanicSchema)

# Get the data into the data frame
dfTitanic_stage = dfReader.options({"field_delimiter":",", "FIELD_OPTIONALLY_ENCLOSED_BY":'"', "NULL_IF":"?", "SKIP_HEADER":1}).csv(f"@{data_stage_name}/titanic.csv.gz")
session.sql("DROP TABLE IF EXISTS titanic").collect()
dfTitanic_stage.copy_into_table("titanic")

In [ ]:
session.table("titanic").show()

### Campaign spend

In [ ]:
# Upload the source file to the stage
session.file.put(f"{data_path}campaign_spend.csv", f"@{data_stage_name}", auto_compress=True, overwrite=True)

In [ ]:
campaignSchema = T.StructType(
    [
        T.StructField("CAMPAIGN", T.StringType()),
        T.StructField("CHANNEL", T.StringType()),
        T.StructField("DATE", T.DateType()),
        T.StructField("TOTAL_CLICKS", T.DecimalType(38,0)),
        T.StructField("TOTAL_COST", T.DecimalType(38,0)),
        T.StructField("ADS_SERVED", T.DecimalType(38,0)),
    ]
)

# Crete a reader
dfReader = session.read.schema(campaignSchema)

# Get the data into the data frame
dfCampaign_stage = dfReader.options({"field_delimiter":",", "SKIP_HEADER":1}).csv(f"@{data_stage_name}/campaign_spend.csv.gz")
session.sql("DROP TABLE IF EXISTS campaign_spend").collect()
dfCampaign_stage.copy_into_table("campaign_spend")

In [ ]:
session.table("campaign_spend").show()

### Bank Marketing files



In [7]:
session.sql("CREATE OR REPLACE STAGE SOURCE_FILES").collect()

session.file.put(f"{data_path}/bank/*", f"@SOURCE_FILES/BANK_MARKETING", auto_compress=False, overwrite=True)

[PutResult(source='data_apr.snappy.parquet', target='data_apr.snappy.parquet', source_size=40357, target_size=40368, source_compression='PARQUET', target_compression='PARQUET', status='UPLOADED', message=''),
 PutResult(source='data_aug.snappy.parquet', target='data_aug.snappy.parquet', source_size=62202, target_size=62208, source_compression='PARQUET', target_compression='PARQUET', status='UPLOADED', message=''),
 PutResult(source='data_dec.snappy.parquet', target='data_dec.snappy.parquet', source_size=8559, target_size=8560, source_compression='PARQUET', target_compression='PARQUET', status='UPLOADED', message=''),
 PutResult(source='data_feb.snappy.parquet', target='data_feb.snappy.parquet', source_size=36471, target_size=36480, source_compression='PARQUET', target_compression='PARQUET', status='UPLOADED', message=''),
 PutResult(source='data_jan.snappy.parquet', target='data_jan.snappy.parquet', source_size=22809, target_size=22816, source_compression='PARQUET', target_compression=

### Clean up

In [ ]:
session.sql(f"rm @{data_stage_name}/campaign_spend.csv.gz").collect()
session.sql(f"rm @{data_stage_name}/titanic.csv.gz").collect()
session.sql(f"ls @{data_stage_name}").show()

In [ ]:
session.close()